# 5.1 계산그래프
그래프:여러개의 노드와 에지로 표현되는 것 

## 5.1.1 계산그래프로 풀다  
순전파:계산을 왼쪽에서 오른쪽으로 진행하는 단계  
역전파:순전파와 반대방향  

## 5.1.2 국소적 계산  
cf)국소적:자신과 직접 관계된 작은 범위  
여기서 각 노드는 결국 자신과 관련된 계산(ex)입력한 두숫자의 덧셈)외에는 신경 쓸 것이 없다  

## 5.1.3 왜 계산그래프로 푸는가?  
### 계산그래프의 이점:
- 국소적 계산  
- 중간계산결과를 모두 보관가능  
- 역전파를 통해 미분을 효율적으로 계산 가능  

# 5.2 연쇄법칙
연쇄법칙:국소적 미분을 전달할때 사용

## 5.2.1 계산 그래프의 역전파  
역전파의 계산 절차는 신호(E)에 노드의 국소적미분을 곱한 후 다음 노드로 전달하는 것  

## 5.2.2 연쇄법칙이란?  
합성함수:여러함수로 구성된 함수  
ex)z=t**2 이때 t=x+y  
합성함수의 미분은 합성함수를 구성하는 각 함수의 미분 곱으로 나타낼 수 있다.  

## 5.2.3 연쇄법칙과 계산 그래프  
역전차가 하는 일은 연쇄법칙의 원리와 같다고 할 수 있다.  

# 5.3 역전파  

## 5.3.1 덧셈노드의 역전파  
z=x+y라는 식을 대상으로 그 역전파를 살펴본다고 가정.x에 대한 z의 미분은 1,y에 대한 z의 미분도 1이다.  
상류로부터 z에 대한 L의 미분값이 전해지고 다시 하류로 x에 대한 L의 미분,y에 대한 L의 미분값을 전달한다.  
if)덧셈노드 가정시(10+5=15이고 상류값이 1.3), 덧셈노드 역전파는 입력신호를 다음 노드로 출력할 뿐이므로 1.3을 그대로 다음 노드에 전달한다.  

## 5.3.2 곱셈노드의 역전파  
if)z=xy이면, x에 대한 z의 미분은 y,y에 대한 z의 미분은 x이다.  
곱셈노드 역전파는 상류값에 순전파때의 입력신호들을 서로 바꾼 값을 곱해서 하류로 보낸다  
곱셈의 역전파는 순방향 입력신호의 값이 필요하다.그래서 곱셈 노드 구현시, 순전파의 입력신호를 변수에 저장  

# 5.4 단순한 계층 구현하기  
가정)계산그래프의 곱셈노드를 mulLayer,덧셈노드를 addLayer라는 이름 사용

## 5.4.1곱셈계층  
forward():순전파  
backward():역전파  

In [1]:
class MulLayer:
    def __init__(self):#변수x와 y초기화(순전파의 입력값 유지하는 목적으로 사용)
        self.x=None
        self.y=None
    def forward(self,x,y):#x,y를 인수로 받고 두 값을 곱해서 반환
        self.x=x
        self.y=y
        out=x*y
        return out
    def backward(self,dout):#상류에 넘어온 미분에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘림
        dx=dout*self.y#x와 y를 바꾼다  
        dy=dout*self.x
        return dx,dy

In [2]:
apple=100
apple_num=2
tax=1.1

#계층들
mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

#순전파
apple_price=mul_apple_layer.forward(apple,apple_num)
price=mul_tax_layer.forward(apple_price,tax)
print(price)

220.00000000000003


In [3]:
#각 변수에 대한 미분-역전파
dprice=1
dapple_price,dtax=mul_tax_layer.backward(dprice)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)
print(dapple,dapple_num,dtax)

2.2 110.00000000000001 200


cf)주의:backward가 받는 인수는 "순전파의 출력에 대한 미분" 값이다.

## 5.4.2 덧셈계층  

In [4]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self,x,y):
        out=x+y
        return out
    
    def backward(self,dout):
        dx=dout*1
        dy=dout*1
        return dx,dy

In [5]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

#계층들
mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()

#순전파
apple_price=mul_apple_layer.forward(apple,apple_num)
orange_price=mul_orange_layer.forward(orange,orange_num)
all_price=add_apple_orange_layer.forward(apple_price,orange_price)
price=mul_tax_layer.forward(all_price,tax)

#역전파
dprice=1
dall_price,dtax=mul_tax_layer.backward(dprice)
dapple_price,dorange_price=add_apple_orange_layer.backward(dall_price)
dorange,dorange_num=mul_orange_layer.backward(dorange_price)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# 5.5활성화 함수 계층 구현하기  
신경망을 구성하는 층(계층)을 클래스 하나씩 구현한다.(ReLu & Sigmoid계층 구현)  

ReLU: 순전파때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 보낸다.  
순전파때가 x가 0이하면 역전파때는 하류로 신호를 보내지 않는다.  

In [19]:
class Relu:
    def __init__(self):
        self.mask=None
        
    def forward(self,x):#순전파
        self.mask=(x<=0)#x가 0이하면 self.mask을 out에 복사해서 0값으로 만들고 아니면 그냥 복사하기 입력값 그대로!
        out=x.copy()
        out[self.mask]=0
        return out
    
    def backward(self,dout):#역전파
        dout[self.mask]=0#상류의 값을 그대로 하류에 전달
        dx=dout
        return dx

In [6]:
import numpy as np
x=np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [7]:
mask=(x<=0)
print(mask)#순전파때 입력이 0이하면 역전파값은 0->역전파때는 순전파때의 mask를 써서 True인 곳은 dout을 0으로 설정

[[False  True]
 [ True False]]


## 5.5.2 시그모이드 계층

1단계 '/' 노드, y = 1 / x를 미분하면 다음식이 됨
역전파 때는 상류의 예측값에 -y**2 을 곱해서 하류로 전달

2단계 상류의 값을 여과 없이 하류로 보냄

3단계 y = exp(x) 연산을 수행
계산 그래프에서는 상류의 순전파 때의 출력(exp(-x))을 곱해 하류로 전파

4단계 y = exp(x) 연산을 수행
'X' 노드, 순전파 때의 값을 서로 바꿔 곱함. 이 경우,-1을 곱함

시그모이드 간소화버전  
노드를 그룹화하여 Sigmoid 계층의 세세한 내용을 노출하지 않고 입력과 출력에만 집중  
Sigmoid 계층의 계산 그래프: 순전파의 출력 y만으로 역전파를 계산  

In [8]:
class Sigmoid:
    def __init__(self):
        self.out=None
        
    def forward(self,x):#순전파
        out=1/(1+np.exp(-x))
        self.out=out
        return out
    
    def backward(self,dout):#역전파
        dx=dout*(1.0-self.out)*self.out
        return dx

# 5.6 Affine/Softmax계층 구현하기  

## 5.6.1 Affine계층  
affine계층:어파인 변화을 수행하는 처리  

주의사항: 어핀계층의 계산 그래프에서 행렬 곱에 주의를 기울인다.  
- 행렬 곱에서 대응하는 차원의 원소수를 일치시켜야 한다.

## 5.6.2 배치용 Affine계층  

입력 X의 형상(shape)이 (N,2)가 됨.  
이후에는 순전파할때는 위와 동일, 역전파때는 행렬의 shape형상에 주의하면서 값 도출  
if)N=2(데이터2개)인 경우, 편향은 두 데이터 각각에 더해진다.  

In [9]:
X_dot_W=np.array([[0,0,0],[10,10,10]])
B=np.array([1,2,3])
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [10]:
X_dot_W+B#순전파의 편향 덧셈은 각각의 데이터 원소에 더해진다.

array([[ 1,  2,  3],
       [11, 12, 13]])

In [11]:
dY=np.array([[1,2,3],[4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

편향의 역전파는 두 데이터에 대한 미분을 데이터마다 더해서 구한다.  

In [12]:
dB=np.sum(dY,axis=0)#np.sum()에서 0번재 축(column)에 대해 총합을 구한다.
dB

array([5, 7, 9])

In [13]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
    
    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b
        return out
    
    def backward(self,dout):
        dx=np.dot(dout,self.W.T)
        self.dW=np.dot(self.x.T,dout)
        self.db=np.sum(dout,axis=0)
        return dx

## 5.6.3 Softmax-with-Loss계층  
소프트맥스함수는 입력값을 정규화해서 출력한다  
ex)Mnist데이터셋에서 손글씨 숫자는 가짓수가 10개(클래스 분류)이므로 softmax계층의 입력이 10개가 된다.  

cf) 신경망에서 수행하는 작업은 '학습' 과 '추론' 이다.  
추론시에는 소프트맥스함수를 사용하지 않고 신경망 추론시에는 마지막 Affine계층의 출력을 인식한다.(학습시에는 사용)  
신경망에서 정규화되지 않는 출력결과에서는 softmax앞의 Affine계층의 출력을 점수(score)라고 한다.  

소프트맥스 계층 구현: 손실 함수인 교차 엔트로피 오차도 포함하여 'Softmax-with-Loss 계층'이라는 이름으로 구현  
Softmax 계층: 입력 (a1, a2, a3)를 정규화 및 (y1, y2, y3)를 출력  
Cross Entropy 계층: Softmax의 출력(y1, y2, y3)과 정답 레이블(t1, t2, t3)를 받고, 손실 L을 출력  
Softmax 계층의 역전파:(y1-t1, y2-t2, y3-t3)->softmax계층의 출력과 정답레이블의 차분  

신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해짐  
소프트맥스 함수의 손실 함수로 교차 엔트로피 오차를 사용하니 역전파가 (y1-t1, y2-t2, y3-t3)로 말끔히 떨어짐  
- 결국 y1-t1,y2-t2...등은 softmax계층의 출력과 정답레이블의 차이로 "신경망의 현재 출력" 과 "정답레이블의 오차"를 나타냄  

In [16]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None#손실
        self.y=None#softmax출력
        self.t=None#정답레이블(원핫벡터)
        
    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size
        return dx

# 5.7 오차역전파법 구현하기

## 5.7.1 신경망 학습의 전체 그림  
전제  
- 신경망에는 적응가능한 가중치, 편향이 존재  
- 가중치와 편향을 훈련데이터에 적응하도록 조정하는 과정='학습'  

1단계-미니배치  
- 훈련데이터 일부를 무작위(미니배치)로 가져온다.  
- 미니배치의 손실함수 값을 줄이는게 목표  

2단계-기울기 산출  
- 미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기 구함  
- (기울기:손실함수의 값을 줄여주는 방향 제시)  

3단계-매개변수 갱신  
- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신  

4단계-반복  
- 1~3단계 반복  

In [17]:
import numpy as np
#from common.layers import *
#from common.gradient import numerical_gradient
from collections import OrderedDict

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 방지
    return np.exp(x) / np.sum(np.exp(x))

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()      
    return grad


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()                                           
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) 
        self.layers['Relu1'] = Relu()                                         
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2']) 
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():                              
            x = layer.forward(x)                                           
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
        
    def gradient(self, x, t):#가중치 매개변수의 기울기를 오차역전법으로 구하기!
        # forward
        self.loss(x, t)                  

        # backward
        dout = 1                             
        dout = self.lastLayer.backward(dout) 
        
        layers = list(self.layers.values())  
        layers.reverse()                     
        for layer in layers:                 
            dout = layer.backward(dout)      

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads

OrderedDict: 딕셔너리에 추가한 순서를 기억하는 (순서가 있는) 딕셔너리  
순전파 때는 추가한 순서대로 각 계층의 forward() 메서드를 호출  
역전파 때는 계층을 반대 순서로 호출  

## 5.7.3 오차역전파법으로 구한 기울기 검증하기  
1. 수치미분으로 써서 구하는 방법  
2. 수식을 해석적으로 풀어서 구하는 방법  
수치미분법은 느리니까 오차역전파법을 이용하는 것이 더 좋음  

In [20]:
from mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch = x_train[:3]
t_batch = t_train[:3]

In [21]:
%timeit network.numerical_gradient(x_batch, t_batch)

9.96 s ± 60.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit network.gradient(x_batch, t_batch)

294 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [24]:
import numpy as np
from mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:2.199807833724237e-13
b1:8.407382158081427e-13
W2:9.163717933069732e-13
b2:1.2057021631095565e-10


결론: 실제로 numerical_gradient는 gradient보다 느리다.

## 5.7.4 오차역전파법을 사용한 학습 구현하기  

In [25]:
import numpy as np
from mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) #수치미분 방식
    grad = network.gradient(x_batch, t_batch) #오차역전파법 방식
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.14103333333333334 0.1445
0.9058833333333334 0.9088
0.9227833333333333 0.9258
0.9329166666666666 0.9329
0.94555 0.9445
0.95175 0.9499
0.95735 0.9551
0.9621666666666666 0.9596
0.9641166666666666 0.9598
0.9643 0.9587
0.9704833333333334 0.9666
0.9729166666666667 0.9662
0.9731333333333333 0.9675
0.9744166666666667 0.9669
0.9764833333333334 0.9702
0.9778166666666667 0.9704
0.9780333333333333 0.9687
